## 4- Daily Summary Table

How to import the daily summary table & demonstrations of what information it has. Slides explaining motivation located [here](https://docs.google.com/presentation/d/1Z_Q5WOAo14Pu6IyJc0vErCub9KArtuX0fUCAMdcbe-A/edit?usp=sharing).

Written by Jess Breda May 2023

### 1.1 Libraries

In [ ]:
from create_summary_table import create_daily_summary_from_dj, lazy_load_daily_summary_df
from plot_summary_table import *

import seaborn as sns
import matplotlib.pyplot as plt

# settings
%load_ext autoreload
%autoreload 2

sns.set_context("talk")

### 1.2 Assumptions

1. This code will grab info for any date when the animal had an entry in the `Sessions` table or the `Mass` table.

2. It is pulling the restriction volume from the `Water` table. This volume only gets entered if pub is run that day. If there is no entry, it defaults to 4% for mice and 3% for rats. To know when this is happening set `Verbose` to `True`.

3. If `AdlibGUI` is not turned on for the animal `Waptor ON`, there will be no rig volume in the RigWater table and it will default to 0. To know when this is happening set `Verbose` to `True`.

4. In rare cases, an animal will have an entry in the Sessions table, but not the `Mass` table. If this happens, the code will grab the weight from 1 day previous. To know when this is happening set `Verbose` to `True`.

### 2. Fetch for a single animal

Create the daily summary dataframe using the `fetch_daily_summary_info` function. If you have the datajoint environment set up (see README), this should run for any animal(s). 

In [ ]:
ANIMAL_ID = ["A325"]


animal_df = create_daily_summary_from_dj(
    animal_ids=ANIMAL_ID, 
    date_min="2023-05-07", 
    date_max="2023-05-25",
    verbose=False,
)

In [ ]:
animal_df.info()

In [ ]:
animal_df.head()

### 3. Visualize

Examples of how to use plotting functions with the daily summary dataframe of a single animal

In [ ]:
# Plot number of done trials and rate of trials/hr over dates

fig,ax = make_fig() # helper function
plot_trials(animal_df, ax, title= ANIMAL_ID[0], legend=True)


In [ ]:
# Plot mass over dates

fig,ax = make_fig()
plot_mass(animal_df, ax, title=ANIMAL_ID[0])


In [ ]:
# Plot water restriction over dates

fig,ax = make_fig()
plot_water_restriction(animal_df, ax, title=ANIMAL_ID[0], legend=False)

In [ ]:
# Plot rig / tech over dates

fig,ax = make_fig()
plot_rig_tech(animal_df, ax, title=ANIMAL_ID[0])


### 4. Extensions

#### 4.1 Multi-plot summary

Here's an example of how to easily make a subplot for an animal using `subplot_mosiac`

In [ ]:

## Initialize mutli-axis figure
layout = """
    AAABBB
    CCCDDD
    EEEFFF
"""
fig = plt.figure(constrained_layout=True, figsize=(15,8))
ax_dict = fig.subplot_mosaic(layout)  # ax to plot to
plt.suptitle(f"\n{ANIMAL_ID[0]} Daily Summary Plot", fontweight="semibold")


## Plot
# left column
plot_trials(animal_df, ax_dict["A"], title="Trials", legend=True, xaxis_label=False)
plot_performance(animal_df, ax_dict["C"], title="Performance", xaxis_label=False)
plot_side_bias(animal_df, ax_dict["E"], title="Side Bias", xaxis_label=True)

# right column
plot_mass(animal_df, ax_dict["B"], title="Mass", xaxis_label=False)
plot_water_restriction(animal_df, ax_dict["D"], title="Water", legend=False, xaxis_label=False)
plot_rig_tech(animal_df, ax_dict["F"], title="Rig/Tech", xaxis_label=True)


#### 4.2 Date window generator

This is handy helper function for generating date windows. For example, I commonly search for 7 days back from the current date. Then, you can only query this date range in datajoint.

In [ ]:
date_min, date_max = return_date_window(latest_date=None, n_days_back=7)
print(f"date min = {date_min}, date max = {date_max}")

In [ ]:
dw_df = create_daily_summary_from_dj(animal_ids=ANIMAL_ID, date_min=date_min, date_max=date_max)
dw_df.head()

In [ ]:
print(
    f"query min: {date_min} returned min: {dw_df.date.min()}," 
    f"\nquery max: {date_max} returned_max: {dw_df.date.max()}"
)

#### 4.3 Multiple animals

The code allows you to fetch multiple animals at a time by passing in a list. You can then use `groupby` functions for plotting

In [ ]:
# Load in 

MULTIANIMAL_IDS = ["R610", "Y510"]

mutlianimal_df = create_daily_summary_from_dj(
    animal_ids=MULTIANIMAL_IDS,
    date_min="2023-05-15", 
    date_max="2023-05-22",
    verbose=False)

In [ ]:
# Check that mutlianimal load in worked
mutlianimal_df.animal_id.unique()

In [ ]:
# Plot average trial rate per animal

fig,ax=make_fig((3,3))
mutlianimal_df.groupby(
    ["animal_id"]
    ).trial_rate.mean().plot(
    kind="bar", title="Mean trial rate"
    )

In [ ]:
## Plot mass over time conditioned on animal id

fig,ax = plt.subplots(1,1, figsize=(10,3))

sns.lineplot(data=mutlianimal_df, x="date", y="mass", hue="animal_id", marker="o", ax=ax)

set_date_x_ticks(ax,xaxis_label=True)
ax.grid()

In [ ]:
# Plot water restriction over time conditioned on animal id

fig,ax = plt.subplots(2,1, figsize=(10,6), sharex=True)

for ii, (animal_id, sub_df) in enumerate(mutlianimal_df.groupby("animal_id")):
    plot_water_restriction(sub_df, ax[ii], title=animal_id, legend=False, xaxis_label=True)


#### 4.4 Overwrite checks

Making the summary data frame is time consuming. To avoid this you can do a "lazy load" and only read in the new dates from bdata and merge this with a dataframe containing the previous dates.

This functionality is implemented with the `overwrite`, `save_out` and `save_path` variables.

In [ ]:
lazy_date_min = "2023-04-06"
lazy_date_max = "2023-05-27"

lazy_load_ids = ["R610", "R611", "R612", "R613", "R614"]

df = lazy_load_daily_summary_df(
    animal_ids=lazy_load_ids, 
    date_min=lazy_date_min, 
    date_max=lazy_date_max, 
    save_out=True
)


In [ ]:
print(f"query min: {lazy_date_min} returned min: {df.date.min()},"
      f"\nquery max: {lazy_date_max} returned_max: {df.date.max()}")


In [ ]:
df